In [92]:
import numpy as np
import matplotlib.pyplot as plt
import cv2 
import random
import math
from datetime import datetime

In [93]:
def add_landmarks_for_face(image, center, mouthCenter, axis1, axis2, angle, size):
    # add landmarks to eyes 
    # Calculate the coordinates of the left and right eye centers
    left_eye_center = (int(center[0] - size[0] * 0.15), int(center[1] - size[1] * 0.1))
    right_eye_center = (int(center[0] + size[0] * 0.15), int(center[1] - size[1] * 0.1))

    # imageLeftEye = cv2.circle(image, center=left_eye_center, radius=5, color=(0, 0, 255), thickness=-1)
    # imageRightEye = cv2.circle(imageLeftEye, center=right_eye_center, radius=5, color=(0, 0, 255), thickness=-1)

    #add landmarks to eyes perimeter
    left_eye_points = []
    for j in range(0, 360, 60):
        xl = int(left_eye_center[0] + size[0] * 0.09 * np.cos(np.deg2rad(j)))
        yl = int(left_eye_center[1] + size[1] * 0.09 * np.sin(np.deg2rad(j)))
        left_eye_points.append((xl, yl))
    
    right_eye_points = []
    for k in range(0, 360, 60):
        xr = int(right_eye_center[0] + size[0] * 0.09 * np.cos(np.deg2rad(k)))
        yr = int(right_eye_center[1] + size[1] * 0.09 * np.sin(np.deg2rad(k)))
        right_eye_points.append((xr, yr))

    # draw the eyes
    # for i, point in enumerate(left_eye_points):
    #     cv2.putText(image, str(i), (point[0] + 5, point[1] + 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1) # for debugging to see the order of points made 
    #     imageLeftEye = cv2.circle(image, point, 5, (0, 0, 255), -1)
    # for i, point in enumerate(right_eye_points):
    #     cv2.putText(imageLeftEye, str(i), (point[0] + 5, point[1] + 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1)
    #     imageRightEye = cv2.circle(imageLeftEye, point, 5, (0, 0, 255), -1)

    perimeter_points = []
    if angle == 0: # happy
      for i in range(0, 170, 30): # Half of the ellipse
        x = int(mouthCenter[0] + axis1 * np.cos(np.deg2rad(i)))
        y = int(mouthCenter[1] + axis2 * np.sin(np.deg2rad(i)))
        perimeter_points.append((x, y)) # curve points
      num_line_points = len(perimeter_points) # so flat part of the ellipse is same len as curved
      line_points = [(x, mouthCenter[1]) for x in range(mouthCenter[0] - axis1, mouthCenter[0] + axis1, int((2 * axis1)/math.floor(num_line_points)))]
      if (len(line_points) != num_line_points): ## sometimes it adds an extra point this just removes that point, its always the last one
        line_points.pop()
      mouth_points = perimeter_points + line_points 
    else:
      for i in range (180, 350, 30): # sad and other half of the ellipse
        x = int(mouthCenter[0] + axis1 * np.cos(np.deg2rad(i)))
        y = int(mouthCenter[1] + axis2 * np.sin(np.deg2rad(i)))
        perimeter_points.append((x, y))
      num_line_points = len(perimeter_points) # so flat part of the ellipse is same len as curved
      line_points = [(x, mouthCenter[1]) for x in range(mouthCenter[0] + axis1, mouthCenter[0] - axis1, -int((2 * axis1) / math.floor(num_line_points)))]
      if (len(line_points) != num_line_points): ## sometimes it adds an extra point this just removes that point, its always the last one
        line_points.pop()
      mouth_points = line_points + perimeter_points 

    # draw the mouth
    # for i, point in enumerate(mouth_points):
    #   cv2.putText(imageRightEye, str(i), (point[0] + 5, point[1] + 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1) # for debugging to see the order of points made 

    #   imageMouth = cv2.circle(imageRightEye, point, 5, (0, 0, 255), -1)
    landmarks = np.array(left_eye_points+ right_eye_points+ mouth_points)

    
    return landmarks #, imageMouth

In [94]:
def create_mask(lefteye, righteye, mouth, size):
    # Create a blank mask with zeros
    mask = np.zeros((size[0], size[1]), dtype=np.uint8)


    # Draw circles for the left and right eyes on the mask
    cv2.circle(mask, center=lefteye, radius=25, color=1, thickness=-1)
    cv2.circle(mask, center=righteye, radius=25, color=1, thickness=-1)
    cv2.fillPoly(mask, [mouth], color=2)

    return mask


In [97]:
size = (320, 256)

def generate_smiley_face(size, num_faces):
  faces = []
  angles = []

  mouthCenter = (int(size[0] // 2), int(size[0] // 2 + size[1] * 0.1))
  for i in range(num_faces):
    center = (size[1] // 2, size[0] // 2)
    angle = random.choice([0 , 180]) 
    if angle == 180:
      mouthCenter = (int(center[0]), int(center[1] + size[1] * 0.15))
    else: # happy
      mouthCenter = (int(center[0]), int(center[1] + size[1] * 0.1))
    axis1 = random.choice([70, 80, 90])
    axis2 = random.choice([25, 35, 40 ])
    image = np.zeros((size[0], size[1], 3), dtype=np.uint8)
    imageCircle = cv2.circle(image, center, size[0] // 3, (255, 255, 255), -1)
    imageLeftEye = cv2.circle(imageCircle, center=(int(center[0] - size[0] * 0.15), int(center[1] - size[1] * 0.1)), radius=25, color=(125, 125, 125), thickness=-1)
    imageRightEye = cv2.circle(imageLeftEye, center=(int(center[0] + size[0] * 0.15), int(center[1] - size[1] * 0.1)), radius=25, color=(125, 125, 125), thickness=-1)
    # imageNose = cv2.circle(imageRightEye, center=(int(center[0]), int(center[1])), radius=10, color=(125, 125, 125), thickness=-1)
    
    imageMouth = cv2.ellipse(img = imageRightEye, center = mouthCenter, 
                             axes = (axis1, axis2), angle = angle, startAngle = 0, 
                             endAngle = 180, color = (125, 125, 125), thickness = -1)
    


    # image_with_landmarks, landmarks = add_landmarks_for_face(imageMouth, center, mouthCenter, axis1, axis2, angle, size)
    landmarks = add_landmarks_for_face(imageMouth, center, mouthCenter, axis1, axis2, angle, size)
    np.save(f"./Eyedata/landmarks/smiley_face{i}.npy", landmarks)
    noisy_image = cv2.GaussianBlur(imageMouth, (5, 5), 0)
    cv2.imwrite(f"./Eyedata/noisy_face/smiley_face{i}.png", noisy_image)

    faces.append(imageMouth)
    # cv2.imwrite(f"./Eyedata/smiley_faces/smiley_face{i}.png", imageMouth)

    # mask = create_mask(landmarks[0], landmarks[1], landmarks[2:], size)

    # np.save(f"./Eyedata/masks/smiley_face{i}.npy", mask)
    # cv2.imwrite(f"./Eyedata/masksPng/smiley_face{i}.png", mask)
    # break
    angles.append(angle)
    np.save(f"./Eyedata/angles/smiley_face_angles.npy", angles)
    

  return faces#, angles


faces = generate_smiley_face(size, 100)



# plt.figure(figsize=(10, 10))
# for i, face in enumerate(faces):
#   plt.subplot(3, 3, i + 1)
#   plt.imshow(face)
#   plt.axis('off')
#   plt.tight_layout()

# plt.show()

#save angle in a file



